In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from scenedataset.scenedataset import SceneDataset
from torchvision import transforms
from torchvision.transforms import _transforms_video as video_transforms

/root/.cache/pypoetry/virtualenvs/scenedataset-mZH_Fr2s-py3.8/lib/python3.8/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/root/.cache/pypoetry/virtualenvs/scenedataset-mZH_Fr2s-py3.8/lib/python3.8/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [3]:
dataset = SceneDataset(
    paths=["/SceneDataset/data/02.mkv", "/SceneDataset/data/01.mkv"],
    show_progress=True,
    transform=transforms.Compose(
        [video_transforms.ToTensorVideo(), transforms.Resize((128, 256))]
    ),
    min_max_len=(15, 25),
    duplicate_threshold=0.01,
    duplicate_metric="mae",
    device="cpu",
    root_dir="/SceneDataset/data/.cache",
    num_workers=1
)


2023-03-02 20:18:28.018 | INFO     | scenedataset.scenedataset:_get_video_paths:548 - Found 2 videos
2023-03-02 20:18:28.062 | INFO     | scenedataset.scenedataset:save_scenes:376 - Scene list file /SceneDataset/data/.cache/content/duplicate_metric_mae/threshold_0.01/_SceneDataset_data_02.mkv.json already up to date.


TypeError: type object argument after ** must be a mapping, not list

In [8]:
dataset.scenes

[[Scene(26, 43, /SceneDataset/data/02.mkv, [26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]),
  Scene(43, 60, /SceneDataset/data/02.mkv, [43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]),
  Scene(81, 102, /SceneDataset/data/02.mkv, [81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101]),
  Scene(102, 122, /SceneDataset/data/02.mkv, [102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121]),
  Scene(122, 142, /SceneDataset/data/02.mkv, [122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141]),
  Scene(142, 162, /SceneDataset/data/02.mkv, [142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161]),
  Scene(162, 182, /SceneDataset/data/02.mkv, [162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181]),
  Scene(182, 202, /SceneDataset/data/02.mkv, [182, 